In [1]:
import sys
import rospy
import moveit_commander

moveit_commander

<module 'moveit_commander' from '/opt/ros/melodic/lib/python2.7/dist-packages/moveit_commander/__init__.pyc'>

In [2]:

rospy.init_node('test_ur', anonymous=True)
moveit_commander.roscpp_initialize(sys.argv)

robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

robot.get_group_names()

['arm', 'effector']

In [3]:
group_name = robot.get_group_names()
move_group = moveit_commander.MoveGroupCommander(group_name[0])

In [4]:
planning_frame = move_group.get_planning_frame()

In [5]:
planning_frame

'world'

In [6]:
eef_link = move_group.get_end_effector_link()

In [7]:
eef_link

'ee_link'

In [14]:

robot.get_current_state()


joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  name: [shoulder_pan_joint, shoulder_lift_joint, elbow_joint, wrist_1_joint, wrist_2_joint,
  wrist_3_joint]
  position: [2.089552364507668, 0.2139880985043776, -1.5266808045702982, -1.8249242334372644, -2.362284175181756, 1.5812897047267302]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "world"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects: []
is_diff: False

In [11]:

move_group.set_named_target("up")
plan = move_group.plan()

In [12]:
move_group.execute(plan, wait=True)

True

In [15]:

def move_joints(move_group, goal):
    move_group.go(goal, wait=True)
    move_group.stop()
    
def get_joint_state(move_group):
    joint_state = move_group.get_current_joint_values()
    print "---- rad value ----"
    print joint_state
    print "---- degree value ---- "
    print [joint*180/math.pi for joint in joint_state]

In [16]:

import math

get_joint_state(move_group)

---- rad value ----
[2.0895523645076715, 0.21398809850437672, -1.5266808045702973, -1.8249242334372644, -2.362284175181755, 1.5812897047267302]
---- degree value ---- 
[119.72253155787136, 12.26061491033051, -87.47236676551489, -104.56045650710229, -135.34891324845736, 90.6012262683298]


In [69]:
goal = [ n * math.pi / 180 for n in [90,-100,90,-100,-90,0]]

move_joints(move_group, goal)
get_joint_state(move_group)

---- rad value ----
[1.5707566881407553, -1.7452433807253875, 1.5708669089615244, -1.7452390045632864, -1.5708069556941426, 5.1434824752227826e-05]
---- degree value ---- 
[89.99772887241213, -99.99507993870819, 90.00404406025666, -99.99482920308932, -90.00060899106766, 0.0029469983782976746]


In [70]:
# cartesin value retrieval

def get_goal_pose(move_group):
    joint_state = move_group.get_current_pose()
    print joint_state

In [97]:

get_goal_pose(move_group)

header: 
  seq: 0
  stamp: 
    secs: 1724
    nsecs: 345000000
  frame_id: "world"
pose: 
  position: 
    x: 0.723573027337
    y: 0.111240730402
    z: 1.55620955204
  orientation: 
    x: 0.768049136044
    y: -0.00683168224924
    z: 0.00497378503076
    w: 0.640335157712


In [164]:

import geometry_msgs.msg

pose_goal = geometry_msgs.msg.Pose()

pose_goal.position.x = 0.562
pose_goal.position.y = 0.550
pose_goal.position.z = 1.37679003904

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

In [166]:
pose_goal.position.x = 0.362
pose_goal.position.y = 0.0
pose_goal.position.z = 1.37679003904 - 0.376

pose_goal.orientation.x = 0.768049136044
pose_goal.orientation.y = -0.00683168224924
pose_goal.orientation.z = 0.00497378503076
pose_goal.orientation.w = 0.640335157712
move_group.set_pose_target(pose_goal)

In [167]:
plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()

In [19]:

import tf

tmp = tf.transformations.euler_from_quaternion([
    pose_goal.orientation.x,
    pose_goal.orientation.y,
    pose_goal.orientation.z,
    pose_goal.orientation.w])

[each*180/math.pi for each in tmp]

[-0.008101290206148783, 59.994323938241884, 89.98596654854087]

In [20]:
tf.transformations.quaternion_from_euler(30*math.pi/180,
                                         60*math.pi/180,
                                         90*math.pi/180)

array([-0.1830127,  0.5      ,  0.5      ,  0.6830127])

In [21]:
[p,q,r] = [each*math.pi/180 for each in [0., 60, 90.]]

x,y,z,w =tf.transformations.quaternion_from_euler(p,q,r)

In [22]:
move_group.set_pose_target(pose_goal)

plan = move_group.go(wait=True)
move_group.stop()
move_group.clear_pose_targets()